In [81]:
import nltk
import pandas as pd
import numpy as np
import numpy_groupies as npg
from numpy.lib import recfunctions
import string
import re
import compute_functions
import mwu_functions
import preprocess_corpus
import process_corpus
punctuation = string.punctuation
from importlib import reload
process_corpus = reload(process_corpus)

https://journals.openedition.org/lexis/6231

In [ ]:
corpus_dict = preprocess_corpus.preprocess_bnc('../mwu_chunkyness/bnc/2.0/bnc_tokenized.txt')
# Make into list, easier to work with

In [ ]:
all_unigrams, all_bigrams = process_corpus.get_ngrams(corpus_dict)

In [ ]:
uni_freqs_pc, uni_freqs = process_corpus.get_frequencies(all_unigrams)
bi_freqs_pc, bi_freqs = process_corpus.get_frequencies(all_bigrams)

In [14]:
unigram_id, bigram_id = process_corpus.get_ids(uni_freqs, bi_freqs)

In [19]:
bigram_freqs_array = process_corpus.get_bigram_info(bi_freqs_pc, unigram_id, bigram_id, by_corpus=True)
bigram_freqs_agg = process_corpus.get_bigram_info(bi_freqs, unigram_id, bigram_id, by_corpus=False)

In [82]:
corpus_props = process_corpus.get_corpus_props(bigram_freqs_array)

In [98]:
bigram_freqs_array

array([('!', '!',      0,      0, 'C', 1,        0),
       ('!', '!',      0,      0, 'H', 1,        0),
       ('!', 'a',      0,  80255, 'J', 1,        1), ...,
       ('♭7th', 'd', 635267, 192402, 'C', 1, 12453927),
       ('♭7th', 'g', 635267, 262716, 'C', 1, 12453928),
       ('♯4/♯11', 'chords', 635268, 165199, 'C', 1, 12453929)],
      dtype=[('first', 'O'), ('second', 'O'), ('first_id', '<i4'), ('second_id', '<i4'), ('corpus', 'O'), ('freq', '<i4'), ('bigram_id', '<i4')])

In [99]:
comp_1 = '!'
comp_2 = "!"
bigram_freqs_per_corpus = bigram_freqs_array
bigram_freqs_total = bigram_freqs_agg
unigram_freqs = uni_freqs

In [100]:
bigram_freq = bigram_freqs_per_corpus[np.logical_and(bigram_freqs_per_corpus['first'] == comp_1, bigram_freqs_per_corpus['second'] == comp_2)]
bigram_props = bigram_freq['freq'] / np.sum(bigram_freq['freq'])
bigram_props = zip(bigram_freq['corpus'], bigram_props)
bigram_props = pd.DataFrame(bigram_props, columns=['corpus', 'ngram_prop'])

In [106]:
bigram_props = pd.merge(corpus_props, bigram_props, on='corpus', how='left').fillna(0)

In [64]:
ngram_corpus_props = dict.fromkeys(corpus_props.keys(), 0)
for corpus, prop in bigram_props:
    ngram_corpus_props[corpus] = prop

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [52]:
ngram_corpus_props = np.array(list(ngram_corpus_props.items()), dtype = {'names':['corpus', 'ngram_prop'], 'formats':[np.dtypes.StrDType, 'f']})

In [60]:
corpus_props = np.array(list(corpus_props.items()), dtype = {'names': ['corpus', 'corpus_prop'], 'formats': [np.dtypes.StrDType, 'f']})

In [61]:
corpus_props

array([('D', 0.00084524), ('K', 0.1146091 ), ('C', 0.17681658),
       ('F', 0.09082857), ('H', 0.19949414), ('G', 0.07913015),
       ('B', 0.06791935), ('E', 0.06830774), ('J', 0.0592792 ),
       ('A', 0.14276995)], dtype=[('corpus', 'O'), ('corpus_prop', '<f4')])

In [ ]:
ngram_corpus_props[bigram_freq['corpus']] = bigram_props # This handles dispersion with bigrams with 0 occurrence in 1 or more corpora
dispersion = compute_functions.get_KLD(ngram_corpus_props, corpus_props)

In [35]:
bigram_freq

array([('hello', "i'm", 293638, 307256, 'A',  1, 5036258),
       ('hello', "i'm", 293638, 307256, 'C',  2, 5036258),
       ('hello', "i'm", 293638, 307256, 'E',  1, 5036258),
       ('hello', "i'm", 293638, 307256, 'F',  4, 5036258),
       ('hello', "i'm", 293638, 307256, 'G',  2, 5036258),
       ('hello', "i'm", 293638, 307256, 'H',  5, 5036258),
       ('hello', "i'm", 293638, 307256, 'J',  3, 5036258),
       ('hello', "i'm", 293638, 307256, 'K', 16, 5036258)],
      dtype=[('first', 'O'), ('second', 'O'), ('first_id', '<i4'), ('second_id', '<i4'), ('corpus', 'O'), ('freq', '<i4'), ('bigram_id', '<i4')])

In [29]:
bigram_props

array([0.09090909, 0.09090909, 0.45454545, 0.09090909, 0.09090909,
       0.09090909, 0.09090909])

Get normalization scores: max and min token freq and type freq

In [ ]:
# First sum across corpora
type_1 = npg.aggregate(bigram_freqs_agg['first_id'], 1)
type_1 = type_1[bigram_freqs_agg['first_id']]
type_2 = npg.aggregate(bigram_freqs_agg['second_id'], 1) 
type_2 = type_2[bigram_freqs_agg['second_id']]
log_type_1 = np.log(type_1)
log_type_2 = np.log(type_2)
max_type_1 = np.max(log_type_1)
min_type_1 = np.min(log_type_1)
max_type_2 = np.max(log_type_2)
min_type_2 = np.min(log_type_2)

min_token = np.min(np.log(bigram_freqs_agg['freq']))
max_token = np.max(np.log(bigram_freqs_agg['freq']))

Token frequency

In [ ]:
# bigram_freq = bigram_freqs_array[np.logical_and(bigram_freqs_array['first'] == test_1, bigram_freqs_array['second'] == test_2)]
# bigram_props = bigram_freq['freq'] / np.sum(bigram_freq['freq'])
# ngram_corpus_props = np.zeros(corpus_sizes.shape[0])
# ngram_corpus_props[bigram_freq['corpus']] = bigram_props # This handles dispersion with bigrams with 0 occurrence in 1 or more corpora
# token_freq = np.sum(bigram_freq['freq']) # Remember to log later. Keep it non-log to use in calculations.

In [ ]:
# x_info = np.log2((ngram_corpus_props / corpus_props))
# x_info[x_info == -np.inf] = 0
# KLD_ngram_corpus = np.sum(x_info)
# dispersion = 1 - (np.power(np.e, KLD_ngram_corpus))

Type frequencies

In [ ]:
# # Need aggregate frequencies for the rest
# freqs_ngram_1 = bigram_freqs_agg[bigram_freqs_agg['first'] == test_1]
# freqs_ngram_2 = bigram_freqs_agg[bigram_freqs_agg['second'] == test_2]

In [ ]:
# typef_1 = freqs_ngram_1.shape[0]
# typef_2 = freqs_ngram_2.shape[0]

In [ ]:
x = np.array([1, 3])
y = np.array([3, 7])

In [ ]:
print(np.sum(x * np.log(x / y)))
print(np.inner(x, np.log(x / y)))

Normalized difference of entropy

In [ ]:
# freqs_ngram_1

In [ ]:
# freqs_1 = freqs_ngram_1['freq']
# freqs_2 = freqs_ngram_2['freq']
# dist_1 = freqs_1 / np.sum(freqs_1)
# dist_2 = freqs_2 / np.sum(freqs_2)
# dist_1_diff = np.delete(freqs_1, np.where(freqs_ngram_1['second'] == test_2))
# dist_1_diff = dist_1_diff / np.sum(dist_1_diff)
# dist_2_diff = np.delete(freqs_2, np.where(freqs_ngram_2['first'] == test_1))
# dist_2_diff = dist_2_diff / np.sum(dist_2_diff)

In [ ]:
# h_1 = (-(np.sum(np.log2(dist_1) * dist_1))) / np.log2(dist_1.shape[0])
# h_1_diff = (-(np.sum(np.log2(dist_1_diff) * dist_1_diff))) / np.log2(dist_1_diff.shape[0])
# h_2 = (-(np.sum(np.log2(dist_2) * dist_2))) / np.log2(dist_2.shape[0])
# h_2_diff = (-(np.sum(np.log2(dist_2_diff) * dist_2_diff))) / np.log2(dist_2_diff.shape[0])
# h_ngram_1 = h_1_diff - h_1
# h_ngram_2 = h_2_diff - h_2

Associations

1 to 2: (Prob 1 and 2 vs prob 1 and other) vs (prob 2 vs prob not 2)

2 to 1: (Prob 1 and 2 vs prob other and 2) vs (prob 1 vs prob not 1)

In [ ]:
# prob_1_2 = token_freq / np.sum(freqs_1)
# prob_1_o = 1 - prob_1_2
# prob_2_1 = token_freq / np.sum(freqs_2)
# prob_o_2 = 1 - prob_2_1 

# prob_1 = np.sum(freqs_1) / np.sum(bigram_freqs_agg['freq'])
# prob_no_1 = 1 - prob_1
# prob_2 = np.sum(freqs_2) / np.sum(bigram_freqs_agg['freq'])
# prob_no_2 = 1 - prob_2

In [ ]:
# KLD_1_2 = prob_1_2 * np.log2(prob_1_2 / prob_2) + prob_1_o * np.log2(prob_1_o / prob_no_2)
# KLD_2_1 = prob_2_1 * np.log2(prob_2_1 / prob_1) + prob_o_2 * np.log2(prob_o_2 / prob_no_1)
# assoc_f = 1 - np.power(np.e, -KLD_1_2)
# assoc_b = 1 - np.power(np.e, -KLD_2_1)


Make result dict

In [ ]:
# mwu_scores = {'ngram': (test_1, test_2), 'first': test_1, 'second': test_2, 'token_freq': np.log(token_freq), 'dispersion': 1 - dispersion, 'type_1': np.log(typef_1), 'type_2': np.log(typef_2), 'entropy_1': h_ngram_1, 'entropy_2': h_ngram_2, 'assoc_f': assoc_f, 'assoc_b': assoc_b}

In [ ]:
# mwu_scores

Make function

In [ ]:
# def get_entropy(array):
#     if len(array) == 0:
#         return 1.0
#     if len(array) == 1:
#         return 0.0
#     else:
#         probs = array / np.sum(array)
#         logs = np.log2(probs)
#         return -np.nansum(probs * logs) / np.log2(array.shape[0])

In [ ]:
bigram_freqs_array

In [ ]:
# def get_bigram_scores(comp_1, comp_2, bigram_freqs_per_corpus = bigram_freqs_array, corpus_props = corpus_props, bigram_freqs_total = bigram_freqs_agg):
#     # By default it looks for variables called bigram_freqs_array, corpus_props, and bigram_freqs_total.
#     bigram_freq = bigram_freqs_per_corpus[np.logical_and(bigram_freqs_array['first'] == comp_1, bigram_freqs_per_corpus['second'] == comp_2)]
#     bigram_props = bigram_freq['freq'] / np.sum(bigram_freq['freq'])
    
#     # Token frequency
#     token_freq = np.sum(bigram_freq['freq'])

#     # Dispersion
#     ngram_corpus_props = np.zeros(corpus_sizes.shape[0])
#     ngram_corpus_props[bigram_freq['corpus']] = bigram_props # This handles dispersion with bigrams with 0 occurrence in 1 or more corpora
#     dispersion = compute_functions.get_KLD(ngram_corpus_props, corpus_props)
    
#     ## Need aggregate frequencies for the rest
#     freqs_ngram_1 = bigram_freqs_agg[bigram_freqs_total['first'] == comp_1]
#     freqs_ngram_2 = bigram_freqs_agg[bigram_freqs_total['second'] == comp_2]

#     # Type frequencies    
#     typef_1 = freqs_ngram_2.shape[0]
#     typef_2 = freqs_ngram_1.shape[0] # Type frequency of slot 1 is the number of rows that have slot 1 as first

#     # Entropies

#     slot1_diff = compute_functions.get_entropy_dif(freqs_ngram_2, comp_1, 'first')
#     slot2_diff = compute_functions.get_entropy_dif(freqs_ngram_1, comp_2, 'second')

#     # Associations
#     # joint probability is conditioned on the unigram frequency of the 1st one
#     prob_1_2 = token_freq / uni_freqs[(comp_1,)]
#     prob_2_1 = token_freq / uni_freqs[(comp_2,)]
#     prob_1 = uni_freqs.freq((comp_1,))
#     prob_2 = uni_freqs.freq((comp_2,))
#     assoc_f = compute_functions.get_KLD(np.array([prob_1_2, 1 - prob_1_2]), np.array([prob_2, 1 - prob_2]))
#     assoc_b = compute_functions.get_KLD(np.array([prob_2_1, 1 - prob_2_1]), np.array([prob_1, 1 - prob_1]))

#     return {'ngram': (comp_1, comp_2), 'first': comp_1, 'second': comp_2, 'token_freq': token_freq, 'dispersion': dispersion, 'type_1': typef_1, 'type_2': typef_2, 'entropy_1': slot1_diff, 'entropy_2': slot2_diff, 'assoc_f': assoc_f, 'assoc_b': assoc_b}

In [ ]:
corpus_props

In [ ]:
old = mwu_functions.get_bigram_scores('c', 'b', bigram_freqs_array, corpus_props, bigram_freqs_agg, uni_freqs)
old

In [ ]:
compute_functions.

In [ ]:
new = get_bigram_scores('c', 'b')
new

In [ ]:
def get_mwu_scores(ngrams):
    # ngrams is a list of ngrams separated by spaces 
    all_scores = []
    i = 0
    for ngram in ngrams:
        print(ngram)
        print(i)
        comps = ngram.split(' ')
        if len(comps) == 2:
            bigram_scores = get_bigram_scores(comps[0], comps[1])
            all_scores.append(bigram_scores)
        else:
            bigram_1_score = get_bigram_scores(comps[0], comps[1])
            bigram_2_score = get_bigram_scores(comps[1], comps[2])
            all_scores.append(bigram_1_score)
            all_scores.append(bigram_2_score)
        i += 1
    return pd.DataFrame(all_scores)

# TODO: something's making MWU scores very uniform (~0.3. Whatsup with that?)

In [ ]:
# get_mwu_scores(['b d', 'c b', 'a c', 'd c b'])

In [ ]:
# get_mwu_scores(['a c'])

In [ ]:
# Testing with whole corpus to validate program
# bigrams_bnc = pd.read_csv('scores/bigram_scores_bnc.csv')
filter_bigs = list(filter(lambda x: x[1]>3, bi_freqs.items()))

In [ ]:
import random
bigram_sample = random.sample(filter_bigs, 10000)
bigram_sample = [bigram[0][0] + ' ' + bigram[0][1] for bigram in bigram_sample]

In [ ]:
bigram_sample

In [ ]:
mwu_score_sample = get_mwu_scores(bigram_sample)

In [ ]:
# bigrams_bnc = bigrams_bnc[bigrams_bnc['token_freq'] > 0.001]
# sampled_bigrams = bigrams_bnc.sample(1000)

In [ ]:
bigram_freqs_agg[bigram_freqs_agg['second'] == 'sgsa']

In [ ]:
mwu_score_sample[mwu_score_sample.entropy_1 < -0.01]

In [ ]:
import matplotlib.pyplot as plt
counts, bins = np.histogram(mwu_score_sample['entropy_1'], bins=100)
plt.stairs(counts, bins)

In [ ]:
mwu_scores_filter = mwu_score_sample.loc[(mwu_score_sample['type_1'] > 3) & (mwu_score_sample['type_2'] > 3)]
mwu_scores_filter = mwu_scores_filter.astype({'token_freq': 'float', 'type_1': 'float', 'type_2': 'float'})
mwu_scores_filter.loc[mwu_scores_filter['entropy_1'] > 0.05, 'entropy_1'] = 0.05
mwu_scores_filter.loc[mwu_scores_filter['entropy_2'] > 0.05, 'entropy_2'] = 0.05
# mwu_scores_filter.loc[:, 'entropy_1'] = np.cbrt(mwu_scores_filter['entropy_1'])
# mwu_scores_filter.loc[:, 'entropy_2'] = np.cbrt(mwu_scores_filter['entropy_2'])
mwu_scores_filter.loc[:, 'entropy_1'] = (mwu_scores_filter['entropy_1'] - mwu_scores_filter['entropy_1'].min()) / (mwu_scores_filter['entropy_1'].max() - mwu_scores_filter['entropy_1'].min())
mwu_scores_filter.loc[:, 'entropy_2'] = (mwu_scores_filter['entropy_2'] - mwu_scores_filter['entropy_2'].min()) / (mwu_scores_filter['entropy_2'].max() - mwu_scores_filter['entropy_2'].min())
mwu_scores_filter.loc[:, 'token_freq'] = np.log(mwu_scores_filter['token_freq'])
mwu_scores_filter.loc[:, 'token_freq'] = (mwu_scores_filter['token_freq'] - mwu_scores_filter['token_freq'].min()) / (mwu_scores_filter['token_freq'].max() - mwu_scores_filter['token_freq'].min())
mwu_scores_filter.loc[:, 'type_1'] = np.log(mwu_scores_filter['type_1'])
mwu_scores_filter.loc[:, 'type_2'] = np.log(mwu_scores_filter['type_2'])
mwu_scores_filter.loc[:, 'type_1'] = (mwu_scores_filter['type_1'] - mwu_scores_filter['type_1'].min()) / (mwu_scores_filter['type_1'].max() - mwu_scores_filter['type_1'].min())
mwu_scores_filter.loc[:, 'type_2'] = (mwu_scores_filter['type_2'] - mwu_scores_filter['type_2'].min()) / (mwu_scores_filter['type_2'].max() - mwu_scores_filter['type_2'].min())
mwu_scores_filter.loc[:, 'type_1'] = 1 - mwu_scores_filter['type_1']
mwu_scores_filter.loc[:, 'type_2'] = 1 - mwu_scores_filter['type_2']

mwu_scores_filter.loc[:, 'dispersion'] = 1 - mwu_scores_filter.loc[:, 'dispersion']

In [ ]:
mwu_scores_weighted = mwu_scores_filter.copy()
mwu_scores_weighted.token_freq = mwu_scores_weighted.token_freq * 0.25
mwu_scores_weighted.dispersion = mwu_scores_weighted.token_freq * 0.25
mwu_scores_weighted.type_1 = mwu_scores_weighted.type_1 * 0.25/5
mwu_scores_weighted.type_2 = mwu_scores_weighted.type_2 * 0.25/5
mwu_scores_weighted.entropy_1 = mwu_scores_weighted.entropy_1 * 0.25/5
mwu_scores_weighted.entropy_2 = mwu_scores_weighted.entropy_2 * 0.25/5
mwu_scores_weighted.assoc_f = mwu_scores_weighted.assoc_f * 0.25
mwu_scores_weighted.assoc_b = mwu_scores_weighted.assoc_b * 0.25/5


In [ ]:
mwu_scores_weighted['mwu_score'] = mwu_scores_weighted.sum(numeric_only=True, axis=1)
mwu_scores_weighted.sort_values(by=['mwu_score'], ascending=False)

In [ ]:
mwu_scores_filter['mwu_score'] = mwu_scores_filter.mean(numeric_only=True, axis=1)

In [ ]:
mwu_scores_filter.sort_values(by='mwu_score', ascending=False)

In [ ]:
import matplotlib.pyplot as plt
counts, bins = np.histogram(mwu_scores_filter['entropy_1'], bins=100)
plt.stairs(counts, bins)

Empirical rule of thumb: difference in entropy ranges between -0.01 and 0.05.

In [ ]:
counts, bins = np.histogram(np.cbrt(mwu_transformed['entropy_2']), bins=100)
plt.stairs(counts, bins)

In [ ]:
mwu_score_sample.loc[mwu_score_sample['entropy_1'] > 0.01]
mwu_scores_filter = mwu_scores_filter[(mwu_scores_filter['entropy_2'] <= 0.01) & (mwu_scores_filter['entropy_2'] > -0.01)]

In [ ]:
mwu_scores_filter['h1_normalized'] = (np.cbrt(mwu_scores_filter['entropy_1']) - np.min(np.cbrt(mwu_scores_filter['entropy_1']))) / (np.max(np.cbrt(mwu_scores_filter['entropy_1'])) - np.min(np.cbrt(mwu_scores_filter['entropy_1'])))

In [ ]:
x = 1 / (1 + np.exp(-mwu_scores_filter[mwu_scores_filter['token_freq'] != -np.inf]['entropy_1']))

In [ ]:
counts, bins = np.histogram(x, bins=100)
plt.stairs(counts, bins)

In [ ]:
counts, bins = np.histogram(mwu_scores_filter[mwu_scores_filter['token_freq'] != -np.inf]['entropy_1'], bins=100)
plt.stairs(counts, bins)

In [ ]:
np.cbrt(mwu_scores_filter['entropy_1'])

In [ ]:
np.arcsinh(mwu_scores_filter[mwu_scores_filter['token_freq'] != -np.inf]['entropy_1'])

In [ ]:
counts, bins = np.histogram(np.cbrt(mwu_scores_filter[mwu_scores_filter['token_freq'] != -np.inf]['entropy_1']), bins=100)
plt.stairs(counts, bins)

In [ ]:
np.log(10 + np.sqrt(10) + 1)

In [ ]:
counts, bins = np.histogram(mwu_scores_filter[mwu_scores_filter['token_freq'] != -np.inf]['h1_normalized'], bins=100)
plt.stairs(counts, bins)

In [ ]:
counts, bins = np.histogram(np.arcsinh(mwu_scores_filter[mwu_scores_filter['token_freq'] != -np.inf]['h2_normalized']), bins=100)
plt.stairs(counts, bins)

In [ ]:
mwu_scores_filter.sort_values(by=['h2_normalized'])

In [ ]:
# def get_mwu_scores_trigram(trigrams):
#     # trigrams is a list of trigrams separated by spaces  
#     all_scores = []
#     for trigram in trigrams:
#         comps = bigram.split(' ')
#         bigram_1_score = get_bigram_scores(comps[0], comps[1])
#         bigram_2_score = get_bigram_scores(comps[1], comps[2])
#         all_scores.append(bigram_1_score)
#         all_scores.append(bigram_2-score)
#     return pd.DataFrame(all_scores)


In [ ]:
this_data = pd.read_csv('jolsvai_et_al_data.txt', sep='\t')

In [ ]:
all_trigrams = [list(nltk.ngrams(trigram.split('_'), 2)) for trigram in set(this_data['Trigram'])]

In [ ]:
all_bigrams = list(set([' '.join(bigram) for phrase in all_trigrams for bigram in phrase]))

In [ ]:
all_bigrams[0]

In [ ]:
all_scores = get_mwu_scores(all_bigrams)
all_scores.to_csv('scratch.csv')
# TODO: all dispersion values pretty high. See what's going on.

In [ ]:
all_scores

In [ ]:
mwu_measures = np.zeros(len(uq_bigrams), dtype={'names': ['first', 'second', 'token_freq', 'dispersion', 'type_1', 'type_2', 'entropy_1', 'entropy_2', 'assoc_forw', 'assoc_backw'], 'formats': [np.dtypes.StrDType, np.dtypes.StrDType, 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']})
mwu_measures['first'] = np.array([gram[0] for gram in uq_bigrams], dtype=np.dtypes.StrDType)
mwu_measures['second'] = np.array([gram[1] for gram in uq_bigrams], dtype=np.dtypes.StrDType)

In [ ]:
mwu_measures_tr = np.zeros(len(uq_trigrams), dtype={'names': ['first', 'second', 'token_freq', 'dispersion', 'type_1', 'type_2', 'entropy_1', 'entropy_2', 'assoc_forw', 'assoc_backw'], 'formats': [np.dtypes.StrDType, np.dtypes.StrDType, 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']})
mwu_measures_tr['first'] = np.array([(gram[0] + ' ' + gram[1]) for gram in uq_trigrams], dtype=np.dtypes.StrDType)
mwu_measures_tr['second'] = np.array([gram[2] for gram in uq_trigrams], dtype=np.dtypes.StrDType)

In [ ]:
trigram_freqs_array['first'] = np.array([str(gram['first']) for gram in all_trigrams_info])
trigram_freqs_array['first_id'] = [bigram_id[gram['first']] for gram in all_trigrams_info]
trigram_freqs_array['second'] = np.array([gram['second'] for gram in all_trigrams_info])
trigram_freqs_array['second_id'] = [unigram_id[gram['second']] for gram in all_trigrams_info]
trigram_freqs_array['corpus'] = np.array([gram['corpus'] for gram in all_trigrams_info])
trigram_freqs_array['freq'] = np.array([gram['freq'] for gram in all_trigrams_info])
trigram_freqs_array['trigram_id'] = np.array([gram['trigram_id'] for gram in all_trigrams_info])

In [ ]:
all_bigrams_agg = []
for gram in bi_freqs.items():
    all_bigrams_agg.append({'first': gram[0][0], 'second': gram[0][1], 'freq': gram[1], 'bigram_id': bigram_id[gram[0]]})
all_trigrams_agg = []
for gram in tri_freqs.items():
    all_trigrams_agg.append({'first': (gram[0][0], gram[0][1]), 'second': gram[0][2], 'freq': gram[1], 'trigram_id': trigram_id[gram[0]]})

In [ ]:
dt = {'names':['first', 'second', 'first_id', 'second_id', 'freq', 'bigram_id'], 'formats':[np.dtypes.StrDType, np.dtypes.StrDType, 'i', 'i', 'i', 'i']}
bigram_freqs_agg = np.zeros(len(all_bigrams_agg), dtype=dt)
bigram_freqs_agg['first'] = np.array([gram['first'] for gram in all_bigrams_agg])
bigram_freqs_agg['first_id'] = [unigram_id[gram['first']] for gram in all_bigrams_agg]
bigram_freqs_agg['second'] = np.array([gram['second'] for gram in all_bigrams_agg])
bigram_freqs_agg['second_id'] = [unigram_id[gram['second']] for gram in all_bigrams_agg]
bigram_freqs_agg['freq'] = np.array([gram['freq'] for gram in all_bigrams_agg])
bigram_freqs_agg['bigram_id'] = np.array([gram['bigram_id'] for gram in all_bigrams_agg])

In [ ]:
dt = {'names':['first', 'second', 'first_id', 'second_id', 'freq', 'trigram_id'], 'formats':[np.dtypes.StrDType, np.dtypes.StrDType, 'i', 'i', 'i', 'i']}
trigram_freqs_agg = np.zeros(len(all_trigrams_agg), dtype=dt)
trigram_freqs_agg['first'] = np.array([str(gram['first']) for gram in all_trigrams_agg])
trigram_freqs_agg['first_id'] = [bigram_id[gram['first']] for gram in all_trigrams_agg]
trigram_freqs_agg['second'] = np.array([gram['second'] for gram in all_trigrams_agg])
trigram_freqs_agg['second_id'] = [unigram_id[gram['second']] for gram in all_trigrams_agg]
trigram_freqs_agg['freq'] = np.array([gram['freq'] for gram in all_trigrams_agg])
trigram_freqs_agg['trigram_id'] = np.array([gram['trigram_id'] for gram in all_trigrams_agg])

Token frequencies, pretty easy, aggregate by bigram frequency

In [ ]:
bigram_frequencies = npg.aggregate(bigram_freqs_array['bigram_id'], bigram_freqs_array['freq'], func='sum')
mwu_measures['token_freq'] = bigram_frequencies
trigram_frequencies = npg.aggregate(trigram_freqs_array['trigram_id'], trigram_freqs_array['freq'], func='sum')
mwu_measures_tr['token_freq'] = trigram_frequencies

Dispersion will be operationalized here using a normalized version of the KL-divergence. KL-D between occurrence in corpus and corpus total (i.e. distance from chance).

In [ ]:
# Get total and probs for each corpus
corpus_totals = npg.aggregate(bigram_freqs_array['corpus'], bigram_freqs_array['freq'], func='sum')
total_words = np.sum(corpus_totals)
corpus_totals = corpus_totals[bigram_freqs_array['corpus']]

In [ ]:
corpus_probs = corpus_totals / total_words
corpus_probs = corpus_probs[bigram_freqs_array['corpus']]
bigram_frequencies_rep = bigram_frequencies[bigram_freqs_array['bigram_id']]
prop_appearing_in_corpus = bigram_freqs_array['freq'] / bigram_frequencies_rep

In [ ]:
KLD = prop_appearing_in_corpus * np.log2(prop_appearing_in_corpus / corpus_probs)
KLD = npg.aggregate(bigram_freqs_array['bigram_id'], KLD)
KLD_norm = 1 - np.power(np.e, -KLD)

mwu_measures['dispersion'] = KLD_norm

In [ ]:
# For trigrams
 
corpus_totals = npg.aggregate(trigram_freqs_array['corpus'], trigram_freqs_array['freq'], func='sum')
total_words = np.sum(corpus_totals)
corpus_totals = corpus_totals[trigram_freqs_array['corpus']]
corpus_probs = corpus_totals / total_words
corpus_probs = corpus_probs[trigram_freqs_array['corpus']]
trigram_frequencies_rep = trigram_frequencies[trigram_freqs_array['trigram_id']]
prop_appearing_in_corpus = trigram_freqs_array['freq'] / trigram_frequencies_rep
KLD = prop_appearing_in_corpus * np.log2(prop_appearing_in_corpus / corpus_probs)
KLD = npg.aggregate(trigram_freqs_array['trigram_id'], KLD)
KLD_norm = 1 - np.power(np.e, -KLD)

mwu_measures_tr['dispersion'] = KLD_norm

Now get the type frequencies of each component. From now on, we'll use only the aggregate array.

In [ ]:
# First sum across corpora
type_1 = npg.aggregate(bigram_freqs_agg['second_id'], 1)
type_2 = npg.aggregate(bigram_freqs_agg['first_id'], 1)

In [ ]:
mwu_measures['type_1'] = type_1[bigram_freqs_agg['second_id']]
mwu_measures['type_2'] = type_2[bigram_freqs_agg['first_id']]

In [ ]:
# trigrams
type_1 = npg.aggregate(trigram_freqs_agg['second_id'], 1)
type_2 = npg.aggregate(trigram_freqs_agg['first_id'], 1)
mwu_measures_tr['type_1'] = type_1[trigram_freqs_agg['second_id']]
mwu_measures_tr['type_2'] = type_2[trigram_freqs_agg['first_id']]

Now the entropies.

In [ ]:
# First, let's do slot 1. We need the marginal frequency of the word in slot 2 to transform the conditional frequencies of slot 1 into conditional probabilities.
slot2_freqs = npg.aggregate(bigram_freqs_agg['second_id'], bigram_freqs_agg['freq'], func='sum')
slot2_freqs = slot2_freqs[bigram_freqs_agg['second_id']]
slot1_freqs = npg.aggregate(bigram_freqs_agg['first_id'], bigram_freqs_agg['freq'], func='sum')
slot1_freqs = slot1_freqs[bigram_freqs_agg['first_id']]

In [ ]:
# Now divide the cond. frequency of 1 by marg. of 2 to get cond. prob.
slot1_cprobs = bigram_freqs_agg['freq'] / slot2_freqs
slot1_inf = slot1_cprobs * np.log2(slot1_cprobs)
slot1_h = npg.aggregate(bigram_freqs_agg['second_id'], slot1_inf, func='sum')
slot1_h = -slot1_h
slot1_h = slot1_h[bigram_freqs_agg['second_id']]
# Then normalize, n() is the same as the type frequency for the other slot
slot1_h = slot1_h / np.log2(mwu_measures['type_1'])
slot1_h = np.nan_to_num(slot1_h)

In [ ]:
slot2_cprobs = bigram_freqs_agg['freq'] / slot1_freqs
slot2_inf = slot2_cprobs * np.log2(slot2_cprobs)
slot2_h = npg.aggregate(bigram_freqs_agg['first_id'], slot2_inf, func='sum')
slot2_h = -slot2_h
slot2_h = slot2_h[bigram_freqs_agg['first_id']]
slot2_h = slot2_h / np.log2(mwu_measures['type_2'])
slot2_h = np.nan_to_num(slot2_h)

In [ ]:
mwu_measures['entropy_1'] = slot1_h
mwu_measures['entropy_2'] = slot2_h

In [ ]:
# trigrams
slot2_freqs = npg.aggregate(trigram_freqs_agg['second_id'], trigram_freqs_agg['freq'], func='sum')
slot2_freqs = slot2_freqs[trigram_freqs_agg['second_id']]
slot1_freqs = npg.aggregate(trigram_freqs_agg['first_id'], trigram_freqs_agg['freq'], func='sum')
slot1_freqs = slot1_freqs[trigram_freqs_agg['first_id']]

slot1_cprobs_tr = trigram_freqs_agg['freq'] / slot2_freqs
slot1_inf = slot1_cprobs_tr * np.log2(slot1_cprobs_tr)
slot1_h = npg.aggregate(trigram_freqs_agg['second_id'], slot1_inf, func='sum')
slot1_h = -slot1_h
slot1_h = slot1_h[trigram_freqs_agg['second_id']]
slot1_h = slot1_h / np.log2(mwu_measures_tr['type_1'])
slot1_h = np.nan_to_num(slot1_h)

slot2_cprobs_tr = trigram_freqs_agg['freq'] / slot1_freqs
slot2_inf = slot2_cprobs_tr * np.log2(slot2_cprobs_tr)
slot2_h = npg.aggregate(trigram_freqs_agg['first_id'], slot2_inf, func='sum')
slot2_h = -slot2_h
slot2_h = slot2_h[trigram_freqs_agg['first_id']]
slot2_h = slot2_h / np.log2(mwu_measures_tr['type_2'])
slot2_h = np.nan_to_num(slot2_h)

mwu_measures_tr['entropy_1'] = slot1_h
mwu_measures_tr['entropy_2'] = slot2_h

Now both unidirectional associations

In [ ]:
slot2_p = npg.aggregate(bigram_freqs_agg['second_id'], bigram_freqs_agg['freq'])
slot1_p = npg.aggregate(bigram_freqs_agg['first_id'], bigram_freqs_agg['freq'])

In [ ]:
slot2_p = slot2_p / np.sum(slot2_p)
slot1_p = slot1_p / np.sum(slot1_p)

In [ ]:
slot2_p = slot2_p[bigram_freqs_agg['second_id']]
slot1_p = slot1_p[bigram_freqs_agg['first_id']]

The degree to which 1 attracts a following 2 is computed as the normalized KLD of the probabilities of 2 when 1 is present from the probabilities of 2 in general

In [ ]:
slot2_given_slot1 = np.log2(slot2_cprobs / slot2_p)
other2_given_slot1 = np.log2((1 - slot2_cprobs) / (1 - slot2_p))
slot2_given_slot1 = np.nan_to_num(slot2_given_slot1, neginf=0)
other2_given_slot1 = np.nan_to_num(other2_given_slot1, neginf=0)

In [ ]:
slot1_given_slot2 = np.log2(slot1_cprobs / slot1_p)
other1_given_slot2 = np.log2((1 - slot1_cprobs) / (1 - slot1_p))
slot1_given_slot2 = np.nan_to_num(slot1_given_slot2, neginf=0)
other1_given_slot2 = np.nan_to_num(other1_given_slot2, neginf=0)

In [ ]:
forward_KLD = (slot2_cprobs * slot2_given_slot1) + (1 - slot2_cprobs) * other2_given_slot1
forward_KLD_n = 1 - np.power(np.e, -forward_KLD)
backward_KLD = (slot1_cprobs * slot1_given_slot2) + (1 - slot1_cprobs) * other1_given_slot2
backward_KLD_n = 1 - np.power(np.e, -backward_KLD)

In [ ]:
mwu_measures['assoc_forw'] = forward_KLD_n
mwu_measures['assoc_backw'] = backward_KLD_n

In [ ]:
slot2_p = npg.aggregate(trigram_freqs_agg['second_id'], trigram_freqs_agg['freq'])
slot1_p = npg.aggregate(trigram_freqs_agg['first_id'], trigram_freqs_agg['freq'])

slot2_p = slot2_p / np.sum(slot2_p)
slot1_p = slot1_p / np.sum(slot1_p)

slot2_p = slot2_p[trigram_freqs_agg['second_id']]
slot1_p = slot1_p[trigram_freqs_agg['first_id']]

slot2_given_slot1 = np.log2(slot2_cprobs_tr / slot2_p)
other2_given_slot1 = np.log2((1 - slot2_cprobs_tr) / (1 - slot2_p))
slot2_given_slot1 = np.nan_to_num(slot2_given_slot1, neginf=0)
other2_given_slot1 = np.nan_to_num(other2_given_slot1, neginf=0)

slot1_given_slot2 = np.log2(slot1_cprobs_tr / slot1_p)
other1_given_slot2 = np.log2((1 - slot1_cprobs_tr) / (1 - slot1_p))
slot1_given_slot2 = np.nan_to_num(slot1_given_slot2, neginf=0)
other1_given_slot2 = np.nan_to_num(other1_given_slot2, neginf=0)

forward_KLD = (slot2_cprobs_tr * slot2_given_slot1) + (1 - slot2_cprobs_tr) * other2_given_slot1
forward_KLD_n = 1 - np.power(np.e, -forward_KLD)
backward_KLD = (slot1_cprobs_tr * slot1_given_slot2) + (1 - slot1_cprobs_tr) * other1_given_slot2
backward_KLD_n = 1 - np.power(np.e, -backward_KLD)

mwu_measures_tr['assoc_forw'] = forward_KLD_n
mwu_measures_tr['assoc_backw'] = backward_KLD_n

Now the normalization procedure

but first filter for token_freq > 3

In [ ]:
mwu_measures = mwu_measures[mwu_measures['token_freq'] > 1]

In [ ]:
mwu_measures['token_freq'] = np.log(mwu_measures['token_freq'])
mwu_measures['token_freq'] = (mwu_measures['token_freq'] - np.min(mwu_measures['token_freq'])) / (np.max(mwu_measures['token_freq']) - np.min(mwu_measures['token_freq']))

In [ ]:
mwu_measures['dispersion'] = (mwu_measures['dispersion'] - np.min(mwu_measures['dispersion'])) / (np.max(mwu_measures['dispersion']) - np.min(mwu_measures['dispersion']))
mwu_measures['dispersion'] = 1 - mwu_measures['dispersion']

In [ ]:
mwu_measures['type_1'] = np.log(mwu_measures['type_1'])
mwu_measures['type_1'] = (mwu_measures['type_1'] - np.min(mwu_measures['type_1'])) / (np.max(mwu_measures['type_1']) - np.min(mwu_measures['type_1']))
mwu_measures['type_1'] = 1 - mwu_measures['type_1']
mwu_measures['type_2'] = np.log(mwu_measures['type_2'])
mwu_measures['type_2'] = (mwu_measures['type_2'] - np.min(mwu_measures['type_2'])) / (np.max(mwu_measures['type_2']) - np.min(mwu_measures['type_2']))
mwu_measures['type_2'] = 1 - mwu_measures['type_2']


In [ ]:
mwu_measures['entropy_1'] = (mwu_measures['entropy_1'] - np.min(mwu_measures['entropy_1'])) / (np.max(mwu_measures['entropy_1']) - np.min(mwu_measures['entropy_1']))
mwu_measures['entropy_1'] = 1 - mwu_measures['entropy_1']
mwu_measures['entropy_2'] = (mwu_measures['entropy_2'] - np.min(mwu_measures['entropy_2'])) / (np.max(mwu_measures['entropy_2']) - np.min(mwu_measures['entropy_2']))
mwu_measures['entropy_2'] = 1 - mwu_measures['entropy_2']

In [ ]:
mwu_measures['assoc_forw'] = (mwu_measures['assoc_forw'] - np.min(mwu_measures['assoc_forw'])) / (np.max(mwu_measures['assoc_forw']) - np.min(mwu_measures['assoc_forw']))
mwu_measures['assoc_backw'] = (mwu_measures['assoc_backw'] - np.min(mwu_measures['assoc_backw'])) / (np.max(mwu_measures['assoc_backw']) - np.min(mwu_measures['assoc_backw']))

In [ ]:
mwu_scores_bi = np.array(mwu_measures[['token_freq', 'dispersion', 'type_1', 'type_2', 'entropy_1', 'entropy_2', 'assoc_forw', 'assoc_backw']].tolist(), dtype='f')
mwu_scores_bi = np.mean(mwu_scores_bi, axis=1)

In [ ]:
mwu_scores_labels = [word1 + ' ' + word2 for word1, word2 in zip(mwu_measures['first'].tolist(), mwu_measures['second'].tolist())]

In [ ]:
bigram_scores = pd.DataFrame({'bigram': mwu_scores_labels, 'mwu_score': mwu_scores_bi, 'token_freq': mwu_measures['token_freq'].tolist(), 'dispersion': mwu_measures['dispersion'].tolist(), 'type_1': mwu_measures['type_1'].tolist(), 'type_2': mwu_measures['type_2'].tolist(), 'entropy_1': mwu_measures['entropy_1'].tolist(), 'entropy_2' : mwu_measures['entropy_2'].tolist(), 'assoc_forw': mwu_measures['assoc_forw'].tolist(), 'assoc_backw': mwu_measures['assoc_backw'].tolist()})

In [ ]:
mwu_measures_tr = mwu_measures_tr[mwu_measures_tr['token_freq'] > 1]

mwu_measures_tr['token_freq'] = np.log(mwu_measures_tr['token_freq'])
mwu_measures_tr['token_freq'] = (mwu_measures_tr['token_freq'] - np.min(mwu_measures_tr['token_freq'])) / (np.max(mwu_measures_tr['token_freq']) - np.min(mwu_measures_tr['token_freq']))

mwu_measures_tr['dispersion'] = (mwu_measures_tr['dispersion'] - np.min(mwu_measures_tr['dispersion'])) / (np.max(mwu_measures_tr['dispersion']) - np.min(mwu_measures_tr['dispersion']))
mwu_measures_tr['dispersion'] = 1 - mwu_measures_tr['dispersion']

mwu_measures_tr['type_1'] = np.log(mwu_measures_tr['type_1'])
mwu_measures_tr['type_1'] = (mwu_measures_tr['type_1'] - np.min(mwu_measures_tr['type_1'])) / (np.max(mwu_measures_tr['type_1']) - np.min(mwu_measures_tr['type_1']))
mwu_measures_tr['type_1'] = 1 - mwu_measures_tr['type_1']
mwu_measures_tr['type_2'] = np.log(mwu_measures_tr['type_2'])
mwu_measures_tr['type_2'] = (mwu_measures_tr['type_2'] - np.min(mwu_measures_tr['type_2'])) / (np.max(mwu_measures_tr['type_2']) - np.min(mwu_measures_tr['type_2']))
mwu_measures_tr['type_2'] = 1 - mwu_measures_tr['type_2']

mwu_measures_tr['entropy_1'] = (mwu_measures_tr['entropy_1'] - np.min(mwu_measures_tr['entropy_1'])) / (np.max(mwu_measures_tr['entropy_1']) - np.min(mwu_measures_tr['entropy_1']))
mwu_measures_tr['entropy_1'] = 1 - mwu_measures_tr['entropy_1']
mwu_measures_tr['entropy_2'] = (mwu_measures_tr['entropy_2'] - np.min(mwu_measures_tr['entropy_2'])) / (np.max(mwu_measures_tr['entropy_2']) - np.min(mwu_measures_tr['entropy_2']))
mwu_measures_tr['entropy_2'] = 1 - mwu_measures_tr['entropy_2']

mwu_measures_tr['assoc_forw'] = (mwu_measures_tr['assoc_forw'] - np.min(mwu_measures_tr['assoc_forw'])) / (np.max(mwu_measures_tr['assoc_forw']) - np.min(mwu_measures_tr['assoc_forw']))
mwu_measures_tr['assoc_backw'] = (mwu_measures_tr['assoc_backw'] - np.min(mwu_measures_tr['assoc_backw'])) / (np.max(mwu_measures_tr['assoc_backw']) - np.min(mwu_measures_tr['assoc_backw']))

mwu_scores_tri = np.array(mwu_measures_tr[['token_freq', 'dispersion', 'type_1', 'type_2', 'entropy_1', 'entropy_2', 'assoc_forw', 'assoc_backw']].tolist(), dtype='f')
mwu_scores_tri = np.mean(mwu_scores_tri, axis=1)

mwu_scores_labels_tr = [word1 + ' ' + word2 for word1, word2 in zip(mwu_measures_tr['first'].tolist(), mwu_measures_tr['second'].tolist())]

trigram_scores = pd.DataFrame({'trigram': mwu_scores_labels_tr, 'mwu_score': mwu_scores_tri, 'token_freq': mwu_measures_tr['token_freq'].tolist(), 'dispersion': mwu_measures_tr['dispersion'].tolist(), 'type_1': mwu_measures_tr['type_1'].tolist(), 'type_2': mwu_measures_tr['type_2'].tolist(), 'entropy_1': mwu_measures_tr['entropy_1'].tolist(), 'entropy_2' : mwu_measures_tr['entropy_2'].tolist(), 'assoc_forw': mwu_measures_tr['assoc_forw'].tolist(), 'assoc_backw': mwu_measures_tr['assoc_backw'].tolist()})

In [ ]:
trigram_scores.to_csv('trigram_scores_bnc.csv', index=False)
bigram_scores.to_csv('bigram_scores_bnc.csv', index=False)

In [ ]:
# Get the mwu scores for each sentence

bigram_scores = pd.read_csv('bigram_scores_bnc.csv')
trigram_scores = pd.read_csv('trigram_scores_bnc.csv')

In [ ]:
# make a dictionary with tuple to mwu scores to get the scores for each gram of the sentence
bigram_dict = [tuple(gram.split(' ')) for gram in bigram_scores['bigram']]
trigram_dict = [tuple(gram.split(' ')) for gram in trigram_scores['trigram']]
bigram_dict = dict(zip(bigram_dict, bigram_scores['mwu_score']))
trigram_dict = dict(zip(trigram_dict, trigram_scores['mwu_score']))

In [ ]:
included_bigrams_sent = [[[bigram_dict.get(bigram, None) for bigram in sent] for sent in corpus] for corpus in all_bigrams_sent]
included_trigrams_sent = [[[trigram_dict.get(trigram, None) for trigram in sent] for sent in corpus] for corpus in all_trigrams_sent]

In [ ]:
sents_big_mwu = [[np.array(sentence)[np.array(sentence) != np.array(None)] for sentence in corpus] for corpus in included_bigrams_sent]
sents_trig_mwu = [[np.array(sentence)[np.array(sentence) != np.array(None)] for sentence in corpus] for corpus in included_trigrams_sent]

In [ ]:
sents_big_mwu[1][0]

In [ ]:
np.mean(np.power(np.diff(sents_big_mwu[1][0]), 2))

In [ ]:
sents_big_mwu_mean = [[np.mean(sentence) for sentence in corpus] for corpus in sents_big_mwu]
sents_trig_mwu_mean = [[np.mean(sentence) for sentence in corpus] for corpus in sents_trig_mwu]

In [ ]:
sents_big_stab =  [[np.sqrt(np.mean(np.power(np.diff(sentence), 2))) for sentence in corpus] for corpus in sents_big_mwu]
sents_trig_stab =  [[np.sqrt(np.mean(np.power(np.diff(sentence), 2))) for sentence in corpus] for corpus in sents_trig_mwu]

In [ ]:
sents_big_std =  [[np.std(sentence) if sentence.size > 0 else np.nan for sentence in corpus] for corpus in sents_big_mwu]
sents_trig_std =  [[np.std(sentence) if sentence.size > 0 else np.nan for sentence in corpus] for corpus in sents_big_mwu]

In [ ]:
sents_big_max =  [[np.max(sentence) if sentence.size > 0 else np.nan for sentence in corpus] for corpus in sents_big_mwu]
sents_trig_max =  [[np.max(sentence) if sentence.size > 0 else np.nan for sentence in corpus] for corpus in sents_trig_mwu]
sents_big_min =  [[np.min(sentence) if sentence.size > 0 else np.nan for sentence in corpus] for corpus in sents_big_mwu]
sents_trig_min =  [[np.min(sentence) if sentence.size > 0 else np.nan for sentence in corpus] for corpus in sents_trig_mwu]

In [ ]:
sents_big_n =  [[sentence.size for sentence in corpus] for corpus in sents_big_mwu]
sents_trig_n =  [[sentence.size for sentence in corpus] for corpus in sents_trig_mwu]

In [ ]:
all_sentences = [[' '.join(sentence) for sentence in corpus] for corpus in all_sents]

In [ ]:
bigram_scores = [[{'sentence': sentence, 'mean_bigram': mean, 'std_bigram': std, 'stab_bigram': stab, 'max_bigram': max_mwu, 'min_bigram': min_mwu, 'n_bigram': n, 'corpus': list(corpus_cats)[i]} for sentence, mean, std, stab, max_mwu, min_mwu, n in zip(all_sentences[i], sents_big_mwu_mean[i], sents_big_std[i], sents_big_stab[i], sents_big_max[i], sents_big_min[i], sents_big_n[i])] for i in range(len(sents_big_mwu))]

In [ ]:
trigram_scores = [[{'sentence': sentence, 'mean_trigram': mean, 'std_trigram': std, 'stab_trigram': stab, 'max_trigram': max_mwu, 'min_trigram': min_mwu, 'n_trigram': n, 'corpus': list(corpus_cats)[i]} for sentence, mean, std, stab, max_mwu, min_mwu, n in zip(all_sentences[i], sents_trig_mwu_mean[i], sents_trig_std[i], sents_trig_stab[i], sents_trig_max[i], sents_trig_min[i], sents_trig_n[i])] for i in range(len(sents_trig_mwu))]

In [ ]:
bigram_scores_df = pd.concat([pd.DataFrame(corpus) for corpus in bigram_scores])
trigram_scores_df = pd.concat([pd.DataFrame(corpus) for corpus in trigram_scores])

In [ ]:
bigram_scores_df.to_csv('sentence_bigram_scores_bnc.csv', index=False)
trigram_scores_df.to_csv('sentence_trigram_scores_bnc.csv', index=False)